# Лабораторная работа №3
###Выполнить реализацию SGD в Numba и сравнить с реализацией в Лаб. Работе №1

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from numpy import linalg as la

import numba
from datetime import datetime

#Градиентный спуск

In [3]:
def GD(
    func: callable, grad: callable, start_params: np.ndarray,
    max_iter: int = 1000, lr: float = 0.1
    ) -> np.array:
    
    params = start_params.copy()
    history = [np.array([params[0], params[1], func(params)])]
    
    for i in range(max_iter):
        params = params - lr * grad(params)
        history.append(np.array([params[0], params[1], func(params)]))

    return np.array(history)

#Функция Химмельблау

In [4]:
def Himmelblau(x):
    return (x[0] * x[0] + x[1] - 11) ** 2 + (x[0] + x[1] * x[1] - 7) ** 2

def Himmelblau_Grad(x):
    return np.array([4 * x[0] * (x[0] * x[0] + x[1] - 11) + 2 * (x[0] + x[1] * x[1] - 7), 4 * x[1] * (x[1] * x[1] + x[0] - 7) + 2 * (x[1] + x[0] * x[0] - 11)])

In [6]:
bounds = np.array([10,-10])
start_time = datetime.now()
history = GD(Himmelblau, Himmelblau_Grad, np.array([10,-10]), max_iter = 1000, lr = 0.001)
print("Himmelblau default sgd\n", datetime.now() - start_time)

Himmelblau default sgd
 0:00:00.012673


#Функция Бута

In [5]:
def Booth(x):
    return (x[0] +2 * x[1] - 7) ** 2 + (2 * x[0] + x[1] - 5) ** 2
def Booth_Grad(x):  
    return np.array([10*x[0] + 8 * x[1] - 34, 10*x[1] + 8 * x[0] - 38])

In [7]:
bounds = np.array([10,-10])
start_time = datetime.now()
history = GD(Booth, Booth_Grad, np.array([10,-10]), max_iter = 1000, lr = 0.001)
print("Booth default sgd\n", datetime.now() - start_time)

Booth default sgd
 0:00:00.015731


#Реализация с Numba

In [8]:
@numba.njit(fastmath=True, debug=True)
def GD_Numba(
    func: callable, grad: callable, start_params: np.ndarray,
    max_iter: int = 1000, lr: float = 0.1
    ) -> np.ndarray:
    params = start_params.copy()
    history = [np.array([params[0], params[1], func(params)])]
    for i in range(max_iter):
        params = params - lr * grad(params)
        history.append(np.array([params[0], params[1], func(params)]))

    return history

In [9]:
@numba.njit(fastmath=True)
def Himmelblau_GPU(x) -> np.float64:
    return (x[0] * x[0] + x[1] - 11) ** 2 + (x[0] + x[1] * x[1] - 7) ** 2

@numba.njit(fastmath=True)
def Himmelblau_Grad_GPU(x) -> np.float64:
    return np.array([4 * x[0] * (x[0] * x[0] + x[1] - 11) + 2 * (x[0] + x[1] * x[1] - 7), 4 * x[1] * (x[1] * x[1] + x[0] - 7) + 2 * (x[1] + x[0] * x[0] - 11)])

In [16]:
@numba.njit(fastmath=True)
def Booth_GPU(x) -> np.float64:
    return (x[0] +2 * x[1] - 7) ** 2 + (2 * x[0] + x[1] - 5) ** 2
@numba.njit(fastmath=True)
def Booth_Grad_GPU(x) -> np.float64:  
    return np.array([10*x[0] + 8 * x[1] - 34, 10*x[1] + 8 * x[0] - 38])

In [12]:
bounds = np.array([10,-10])
start_time = datetime.now()
history = GD_Numba(Himmelblau_GPU, Himmelblau_Grad_GPU, np.array([10.,-10.]), max_iter = 1000, lr = 0.001)
print("Himmelblau Numba sgd\n", datetime.now() - start_time)

Himmelblau Numba sgd
 0:00:00.001692


In [18]:
bounds = np.array([10,-10])
start_time = datetime.now()
history = GD_Numba(Booth_GPU, Booth_Grad_GPU, np.array([10.,-10.]), max_iter = 1000, lr = 0.001)
print("Booth Numba sgd\n", datetime.now() - start_time)

Booth Numba sgd
 0:00:00.001213
